In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install --extra-index-url https://artifactory.vgt.vito.be/api/pypi/python-packages/simple terracatalogueclient
# !{sys.executable} -m pip install pip rasterstats

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os, requests, json, geojson
import random, scipy
from collections import defaultdict
from scipy.optimize import curve_fit
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from shapely.ops import unary_union
import rasterio
from rasterio.features import shapes
import shapely
from shapely.ops import transform
from rasterio.merge import merge as rmerge
import rasterio.mask
from rasterstats import point_query
import pandas as pd
import geopandas as gpd
import ee
import geemap
from terracatalogueclient import Catalogue as Terracat
import boto3

In [ ]:
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

In [ ]:
OUTPUT_FILENAME = 'BIO-3-BirdsinBuiltupAreas.csv'

In [ ]:
API_URL = 'https://api.gbif.org/v1/occurrence/search/'
DATASETKEY = '50c9509d-22c7-4a22-a47d-8c48425ef4a7'  # iNaturalist research-grade observations
#TAXONKEYS = {'Arthropoda': '54', 'Aves': '212', 'Tracheophyta': '7707728'}
TAXON = 'Aves'
TAXON_KEY = '212'
STARTYEAR = '2016'
ENDYEAR = '2021'
LIMIT = 300

NUM_CURVEFITS = 200

In [ ]:
worldcover_catalogue = Terracat().authenticate() 
terracat = worldcover_catalogue.authenticate_non_interactive('tedwongwri', 'WRIpass12!')

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

In [ ]:
def boundingbox_wkt(p):
    # Returns WKT for bounding box.
    # Necessary because GBIF API won't accept complex polygons.
    minx, miny, maxx, maxy = p.bounds
    return 'POLYGON (({0} {3}, {0} {2}, {1} {2}, {0} {3}))'.format(str(minx), str(maxx), str(miny), str(maxy))

In [ ]:
def classify_builtup(r):
    r[r != 50] = 0
    r[r == 50] = 1    # built up
    return r

In [ ]:
# This function clips and masks raster
# Adapted from https://gis.stackexchange.com/a/387772

def mask_raster_with_geometry(raster, transform, shapes, **kwargs):
    """Wrapper for rasterio.mask.mask to allow for in-memory processing.

    Docs: https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html

    Args:
        raster (numpy.ndarray): raster to be masked with dim: [H, W]
        transform (affine.Affine): the transform of the raster
        shapes, **kwargs: passed to rasterio.mask.mask

    Returns:
        masked: numpy.ndarray or numpy.ma.MaskedArray with dim: [H, W], and new affine transform
    """
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(
            driver='GTiff',
            height=raster.shape[0],
            width=raster.shape[1],
            count=1,
            dtype=raster.dtype,
            transform=transform,
        ) as dataset:
            dataset.write(raster, 1)
        with memfile.open() as dataset:
            output, new_transform = rasterio.mask.mask(dataset, shapes, **kwargs)
    return output.squeeze(0), new_transform

In [ ]:
def get_birdcount(poly, builtup_raster, builtupraster_transform):
    outputs = []
    # Get observation records from GBIF
    
    observations = []
    
    if poly.type == 'MultiPolygon':
        poly = unary_union(poly)
    
    if str(poly) != 'GEOMETRYCOLLECTION EMPTY':
        box = boundingbox_wkt(poly)

        offset = -LIMIT
        while offset == -LIMIT or not results['endOfRecords']:
            offset += LIMIT
            url = '{0}?dataset_key={1}&taxon_key={2}&year={3},{4}&geometry={5}&limit={6}&offset={7}&hasCoordinate=true'.format(API_URL, DATASETKEY, TAXON_KEY, STARTYEAR, ENDYEAR, box, LIMIT, offset)
            resp = requests.get(url)
            results = resp.json()
            print('  {0}: {1}/{2}'.format(TAXON, results['offset'], results['count']))
            # Note spatial subsetting of points happens below (twice) as part of the conditions in the list comprehensions
            observations += [{
                'species': i['species'],
                'lat': i['decimalLatitude'],  # We don't really need to save lat/lon for this
                'lon': i['decimalLongitude'],
            } for i in results['results'] if 'species' in i.keys() and Point(float(i['decimalLongitude']), float(i['decimalLatitude'])).within(poly)]
        observations = [j for j in observations if point_query(Point((j['lon'], j['lat'])), builtup_raster, affine=builtupraster_transform)[0] != 0]
        # Estimate species counts by estimating asymptote of species-accumulation curve created when observation order is randomized
        # Final estimate is average over NUM_CURVEFITS estimates

        #count_estimate = None
        if len(observations) > 1:
            taxon_observations = [i['species'] for i in observations]
            asymptotes = []
            tries = 0
            while len(asymptotes) < NUM_CURVEFITS:   # Different observation-orders give different results, so average over many
                tries += 1
                taxon_observations.sort(key=lambda x: random.random())                    # Randomize order of observations
                sac = []                                                            # Initialize species accumulation curve data
                for obs_count in range(1, len(taxon_observations)):                       # Go through observation list from beginning
                    sac.append(len(set(taxon_observations[:obs_count])))                  # and count unique species from start to index
                if len(sac) > 5 and tries <= 1000:          # Avoid letting infinite-species errors stop the process
                    try:
                        asymptotes.append(scipy.optimize.curve_fit(lambda x,a,b,c: -((a*np.exp(-b*x))+c), list(range(1, len(sac)+1)), sac)[0][2])
                    except:
                        pass
                else:
                    asymptotes.append(-1)
            if -1 in asymptotes:
                count_estimate = -9999
            else:
                count_estimate = -round(np.mean(asymptotes))
            return count_estimate
    return -9999

In [ ]:
def do_one_geom(row):
    poly = row[0]
    if poly.type == 'Polygon':
        poly = MultiPolygon([poly])
    
    box = boundingbox_wkt(poly)
    products = terracat.get_products('urn:eop:VITO:ESA_WorldCover_10m_2020_V1', geometry=box)
    
    # Create mosaic raster of Worldcover rasters downloaded from ESA
    rasters_to_mosaic = []
    for product in products:
        if not '{0}dir'.format(product.title) in os.listdir('.'):
            terracat.download_file(product.data[0], '{0}dir'.format(product.title))
        raster = rasterio.open('{0}dir/{0}_Map.tif'.format(product.title))
        rasters_to_mosaic.append(raster)

    mosaic, mosaic_transform = rmerge(rasters_to_mosaic)
    
    # Clip raster to district boundary
    clipped_raster, clip_transform = mask_raster_with_geometry(mosaic[0], mosaic_transform, poly, crop=True)
    # Classify clipped raster as habitat/nonhabitat
    builtup_raster = classify_builtup(clipped_raster)
    
    birdcount = get_birdcount(poly, builtup_raster, clip_transform)
    return birdcount

In [ ]:
results = []
for i in range(34,len(boundary_georef)):
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            print(boundary_id)
            boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
            boundary_geo = requests.get(boundary_path).jsoboundary_geo = requests.get(boundary_path).json()
            gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            gdf['BIO_3_percentBirdsinBuiltupAreas'] = gdf.apply(do_one_geom, axis=1)
            results.append(gdf[['geo_id', 'geo_name', 'BIO_3_percentBirdsinBuiltupAreas']].copy())
            output = pd.concat(results, axis=0)
            output.to_csv(OUTPUT_FILENAME)
            # Delete WorldCover rasters to prevent filling up disk
    for directoryname in os.listdir('.'):
        if directoryname[:15] == 'ESA_WorldCover_':
            for filename in os.listdir(directoryname):
                os.remove('{0}/{1}'.format(directoryname, filename))
            os.rmdir(directoryname)

In [ ]:
outdf = result[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'BIO_3_percentBirdsinBuiltupAreas']]
outdf

In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
processedcities

In [ ]:
# csv1 = pd.read_csv('BIO-3-BirdsinBuiltupAreas.csv')
# csv2 = pd.read_csv('BIO-3-BirdsinBuiltupAreas2.csv')

# processedcities = csv1.append(csv2)
# processedcities

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir +'/'+ indicators_file_aws)
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'BIO_3_percentBirdsinBuiltupAreas')

In [ ]:
cities_indicators_merged

## Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = indicators_file_aws
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')